In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Path to dataset
dataset_path = '/content/drive/MyDrive/PlantVillage1'

In [6]:
# Load dataset with TensorFlow's utility
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(256, 256),  # Resize all images to 256x256
    batch_size=32,
    label_mode="int"  # Labels are integers
)

Found 2152 files belonging to 3 classes.


In [7]:
# View class names (folder names are the labels)
class_names = dataset.class_names
print("Class names:", class_names)

Class names: ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']


In [8]:
# Convert dataset to a DataFrame-like structure for analysis
image_paths = []
labels = []


In [9]:
for batch, label_batch in dataset.unbatch():
    image_paths.append(batch.numpy())
    labels.append(label_batch.numpy())

In [10]:
# Convert to NumPy arrays
image_data = np.array(image_paths)
label_data = np.array(labels)

In [11]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    image_data, label_data, test_size=0.2, stratify=label_data, random_state=42
)

print("Training data shape:", X_train.shape, y_train.shape)
print("Testing data shape:", X_test.shape, y_test.shape)

Training data shape: (1721, 256, 256, 3) (1721,)
Testing data shape: (431, 256, 256, 3) (431,)


#CNN Model for feature extraction

In [12]:
from tensorflow.keras import layers, models, regularizers

model = models.Sequential([
    # Convolutional Block 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),

    # Convolutional Block 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Convolutional Block 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Convolutional Block 4
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.4),

    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(len(class_names), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 254, 254, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 125, 125, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 60, 60, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 28, 28, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      12,845,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │              

 Total params: 13,237,443 (50.50 MB)

 Trainable params: 13,235,971 (50.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)


In [14]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6
)


In [15]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 63s 798ms/step - accuracy: 0.7895 - loss: 0.6349 - val_accuracy: 0.5081 - val_loss: 10.9016 - learning_rate: 0.0010
Epoch 2/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 430ms/step - accuracy: 0.9409 - loss: 0.1552 - val_accuracy: 0.5035 - val_loss: 10.5239 - learning_rate: 0.0010
Epoch 3/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 419ms/step - accuracy: 0.9511 - loss: 0.1686 - val_accuracy: 0.6845 - val_loss: 1.2272 - learning_rate: 0.0010
Epoch 4/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 40s 422ms/step - accuracy: 0.9495 - loss: 0.1437 - val_accuracy: 0.7448 - val_loss: 1.5793 - learning_rate: 0.0010
Epoch 5/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 42s 416ms/step - accuracy: 0.9498 - loss: 0.1339 - val_accuracy: 0.6032 - val_loss: 3.2334 - learning_rate: 0.0010
Epoch 6/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 441ms/step - accuracy: 0.9664 - loss: 0.0973 - val_accuracy: 0.8028 - val_loss: 0.9440 - learning_rate: 0.0010
Epoch 7/30
54/54 ━━━━━━━━━━━━━━━━━━━━ 27s 445ms/step - accuracy: 0.9596 - loss: 0.1243 - va

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_pred = np.argmax(model.predict(X_test), axis=-1)
print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
                       precision    recall  f1-score   support

Potato___Early_blight       0.97      0.99      0.98       200
 Potato___Late_blight       0.98      0.96      0.97       200
     Potato___healthy       0.93      0.90      0.92        31

             accuracy                           0.97       431
            macro avg       0.96      0.95      0.96       431
         weighted avg       0.97      0.97      0.97       431

[[199   1   0]
 [  6 192   2]
 [  0   3  28]]


In [17]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9652 - loss: 0.0868
Test Accuracy: 0.9721577763557434


In [18]:
def get_penultimate_layer_index(model):

    return len(model.layers) - 4

In [19]:
penultimate_index = get_penultimate_layer_index(model)
print("Penultimate layer index:", penultimate_index)

# Access the penultimate layer
penultimate_layer = model.layers[penultimate_index]
print("Penultimate layer:", penultimate_layer.name)

Penultimate layer index: 17
Penultimate layer: dense


In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

#  the layer index of the Dense(256) layer
penultimate_layer_index = len(model.layers) - 4  # 3rd last layer is Dense(256)

# new input layer with the correct shape
new_input = Input(shape=(256, 256, 3))

# Connect the new input layer to the first layer of the original model
x = model.layers[0](new_input)

# Connect the rest of the layers sequentially
for layer in model.layers[1:penultimate_layer_index + 1]:
    x = layer(x)

# Create the feature extraction model
# Use new_input as input instead of model.input
feature_extractor = Model(inputs=new_input, outputs=x)
features = feature_extractor.predict(X_train)  # Extract features from the training data
features_flat = features.reshape(features.shape[0], -1)


print("Extracted features shape:", features.shape)  # Should be (100, 256) for Dense(256)

54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step
Extracted features shape: (1721, 256)


In [21]:
pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.0 MB/s eta 0:00:00


In [23]:
from deap import base, creator, tools, algorithms
import random

# Define the problem (maximize accuracy or minimize loss)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
features_flat = features.reshape(features.shape[0], -1)
# GA Setup
toolbox = base.Toolbox()
n_features = features_flat.shape[1]

# Binary encoding for feature selection
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define the evaluation function
def evaluate(individual):
    selected_features = features_flat[:, np.array(individual) == 1]
    if selected_features.shape[1] == 0:
        return 0,  # Avoid division by zero
    # Train a simple classifier on selected features
    num_dummy_samples = features_flat.shape[0]  # Number of samples in the dummy data
    dummy_labels = labels[:num_dummy_samples]  # Assuming labels are already available
    from sklearn.model_selection import cross_val_score
    from sklearn.ensemble import RandomForestClassifier

    clf = RandomForestClassifier()
    scores = cross_val_score(clf, selected_features, y_train, cv=5)  # Replace `labels` with your target labels
    return scores.mean(),

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run GA
population = toolbox.population(n=50)
ngen = 40  # Number of generations
cxpb, mutpb = 0.5, 0.2  # Crossover and mutation probabilities

algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None, halloffame=None, verbose=True)

# Extract the best solution
best_individual = tools.selBest(population, k=1)[0]
optimized_features = features_flat[:, np.array(best_individual) == 1]
print("Optimized features shape:", optimized_features.shape)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	50    
1  	30    
2  	30    
3  	21    
4  	24    
5  	36    
6  	35    
7  	34    
8  	27    
9  	27    
10 	24    
11 	26    
12 	28    
13 	33    
14 	23    
15 	33    
16 	31    
17 	34    
18 	27    
19 	29    
20 	33    
21 	25    
22 	26    
23 	22    
24 	22    
25 	29    
26 	32    
27 	23    
28 	26    
29 	35    
30 	25    
31 	30    
32 	34    
33 	25    
34 	30    
35 	33    
36 	28    
37 	33    
38 	33    
39 	36    
40 	32    
Optimized features shape: (1721, 120)


In [37]:
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
y_train_onehot = to_categorical(y_train, num_classes=len(class_names))

# Create a new model for training with optimized features
optimized_model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(optimized_features.shape[1],)),  # Reduced neurons, added L2 regularization
    Dropout(0.5),  # Added Dropout
    Dense(len(class_names), activation='softmax')
])

# Compile the optimized model
optimized_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Use categorical_crossentropy for one-hot encoded labels
    metrics=['accuracy']
)

# Train with optimized features using the new model and one-hot encoded labels
optimized_model.fit(optimized_features, y_train_onehot, epochs=10, batch_size=32)

y_train_pred_probs = optimized_model.predict(optimized_features)  # Get predicted probabilities
y_train_pred = np.argmax(y_train_pred_probs, axis=1)  # Convert probabilities to class labels

# 2. Calculate accuracy:
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.7669 - loss: 10.8164
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9632 - loss: 1.8401
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9643 - loss: 1.6168
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9843 - loss: 1.1399
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9857 - loss: 1.1511
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9864 - loss: 1.0499
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9867 - loss: 1.0683
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9862 - loss: 1.0478
Epoch 9/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9917 - loss: 0.9669
Epoch 10/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9951 - loss: 0.8551
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training Accuracy: 0.9924462521789658


In [39]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Extract features from X_test using the feature_extractor
test_features = feature_extractor.predict(X_test)
test_features_flat = test_features.reshape(test_features.shape[0], -1)

# Select the optimized features from the extracted test features
optimized_test_features = test_features_flat[:, np.array(best_individual) == 1]

# Now predict using the optimized model and the optimized test features
y_pred = np.argmax(optimized_model.predict(optimized_test_features), axis=-1)

print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
                       precision    recall  f1-score   support

Potato___Early_blight       0.96      0.99      0.98       200
 Potato___Late_blight       0.99      0.95      0.97       200
     Potato___healthy       0.94      0.97      0.95        31

             accuracy                           0.97       431
            macro avg       0.96      0.97      0.97       431
         weighted avg       0.97      0.97      0.97       431

[[199   1   0]
 [  8 190   2]
 [  0   1  30]]


Model is not overfitting for any classes